In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

//anaconda/envs/comet/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Data from [Kaggle](https://inclass.kaggle.com/c/comet-track-recognition-ysda-2016/data) are temporarily available via cernbox. Let's get it.

In [2]:
%%bash
if [ ! -d data ] ; then 
    wget https://cernbox.cern.ch/index.php/s/OuxEIMWpvA4ZlS7/download -O data.tgz && tar xzf data.tgz 
fi
ls -al data

total 990760
drwxr-xr-x  6 atsky  staff        204 13 дек 13:51 .
drwxr-xr-x  7 atsky  staff        238 13 дек 13:55 ..
-rw-r--r--@ 1 atsky  staff   33766823 22 сен 19:47 sample_submission.csv
-rw-r--r--@ 1 atsky  staff  237873474 22 сен 20:01 test.csv
-rw-r--r--@ 1 atsky  staff  235515093 22 сен 20:00 train.csv
-rw-r--r--@ 1 atsky  staff     106657 22 сен 19:44 wires.csv


In [3]:
hits_train = pd.read_csv("data/train.csv", index_col='global_id')
hits_train.head()

,event_id,wire_id,energy_deposit,relative_time,label
global_id,,,,,
0,0,0,0.000000e+00,0.000000,0
1,0,1,0.000000e+00,0.000000,0
2,0,2,0.000000e+00,0.000000,0
3,0,3,0.000000e+00,0.000000,0
4,0,4,1.178108e-08,22.224176,2


In [4]:
hits_test = pd.read_csv("data/test.csv", index_col='global_id')
hits_test.head()

,event_id,wire_id,energy_deposit,relative_time
global_id,,,,
7619400,1700,0,0.000000,0.000000
7619401,1700,1,0.000000,0.000000
7619402,1700,2,0.000000,0.000000
7619403,1700,3,0.000061,515.932708
7619404,1700,4,0.000000,0.000000


# Naive machine learning

In [5]:
from sklearn.tree import DecisionTreeClassifier

In [6]:
wires = pd.read_csv('data/wires.csv')
wire_rho = wires["wire_rho"]
wire_phi = wires["wire_phi"]
neibours = [None] * len(wire_rho)
 
for layer in set(wire_rho):
    p = []

    for j in range(len(wire_rho)):
        if layer != wire_rho[j]: continue
        
        p.append((wire_phi[j], j))
        
    p.sort()
    
    num = len(p)
    for i in range(len(p)):
        prv = (i - 1 + num) % num
        nxt = (i + 1) % num
        neibours[p[i][1]] = (p[prv][1], p[nxt][1])   

In [7]:
def get_data(hits):
    event_id = hits['event_id'].values
    wire_id = hits['wire_id'].values
    energy_deposit = hits['energy_deposit'].values
    relative_time = hits['relative_time'].values
    
    event_list = list(set(event_id))
    events_number = len(event_list)
    event_map = {}
    for e in range(events_number):
        event_map[event_list[e]] = e
    wires_number = 4482
    index = np.zeros((events_number, wires_number), dtype="int") 
    
    num = len(event_id)
    result = np.zeros((num, 7))
            
    for i in range(num):
        index[event_map[event_id[i]], wire_id[i]] = i + 1
            
    print "Index created"
        
    for i in range(num):
        if energy_deposit[i] > 0.0:
            current_e = event_map[event_id[i]]
            current_w = int(wire_id[i])
            
            prv_wire = neibours[current_w][0]
            prv_i = index[current_e, prv_wire] - 1
            nxt_wire = neibours[current_w][1]
            next_i = index[current_e, nxt_wire] - 1
            
            result[i,0] = np.log(energy_deposit[i]) 
            result[i,1] = relative_time[i]
            result[i,2] = wire_rho[current_w]
            result[i,3] = np.log(energy_deposit[prv_i] + 1e-20)
            result[i,4] = relative_time[i] - relative_time[prv_i] 
            result[i,5] = np.log(energy_deposit[next_i] + 1e-20)
            result[i,6] = relative_time[i] - relative_time[next_i]
    
    print "Done"
            
    return result

In [8]:
hits_train_filtered = hits_train.loc[hits_train.energy_deposit > 0]

train_data = get_data(hits_train_filtered)
train_data

Index created
Done


array([[  -18.25677136,    22.22417579,    53.        , ...,
         -727.96249061,   -13.1816232 ,  -727.96249061],
       [  -13.84254559,   101.30041514,    53.        , ...,
         -648.88625126,    -9.74278313,   -33.04443442],
       [   -9.74278313,   134.34484956,    53.        , ...,
           33.04443442,   -13.1816232 ,  -615.84181685],
       ..., 
       [  -13.80508795,  1012.26572774,    80.2       , ...,
          262.07906133,   -12.39993925,   196.67705705],
       [  -12.39993925,   815.58867069,    80.2       , ...,
         -196.67705705,   -13.1816232 ,    65.40200428],
       [  -13.1816232 ,   750.18666641,    80.2       , ...,
          -65.40200428,   -13.1816232 ,     0.        ]])

In [9]:
from sklearn.cross_validation import cross_val_score
cv_entropy = cross_val_score(DecisionTreeClassifier(criterion='entropy'),
                train_data, (hits_train_filtered.label == 1).values.astype(np.int),
                scoring='roc_auc')
print(cv_entropy.mean(), cv_entropy.std())

//anaconda/envs/comet/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(0.97703907993563632, 0.0002120902496683719)


CV might take some time

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import cross_val_score
cv_gini = cross_val_score(GradientBoostingClassifier(),
                train_data, (hits_train_filtered.label == 1).values.astype(np.int),
               scoring='roc_auc')
print(cv_gini.mean(), cv_gini.std())

(0.996717733662984, 0.0001636920281464552)


In [ ]:
classifier = GradientBoostingClassifier()
classifier.fit(train_data, (hits_train_filtered.label == 1))

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [ ]:
candidates = hits_test.loc[hits_test.energy_deposit > 0]
ml_prediction = pd.DataFrame({
        "prediction": classifier.predict_proba(get_data(candidates))[:, 1]
    }, index=candidates.index)

Index created


In [ ]:
ml_prediction.to_csv("naive_ml_prediction.csv", index_label='global_id')

Moral: sometimes you can outdo simple machine learning by thinking. Corollary: the best result is achieved by combining the approaches.

In [ ]:
the_event = hits_train[hits_train.event_id==54]
fig, ax = plt.subplots(figsize=(20,20))
colormap = 'spectral'
wires = pd.read_csv('data/wires.csv')

wires_cartesian = np.vstack((wires['wire_rho'] * np.cos(wires['wire_phi']),
                                  wires['wire_rho'] * np.sin(wires['wire_phi']))).T

ax.scatter(wires_cartesian[:, 0], wires_cartesian[:, 1], c=2-the_event.label, edgecolors='none',
           s=100, cmap=colormap)
# We want to know what color corresponds to which label
labels_x = (-20, 0, 20)
ax.scatter(labels_x, (0, 0, 0), c=(2, 1, 0), cmap=colormap, edgecolors='none', s=300)
for label, coordinate in zip(("0, inactive", "1, signal", "2, noise"), labels_x):
    ax.annotate(label, xy=(coordinate-4, 3))


### Energy deposits

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))
ax.scatter(wires_cartesian[:, 0], wires_cartesian[:, 1], c=np.log(the_event.energy_deposit), edgecolors='none',
           s=100, cmap='bwr')